In [ ]:
# Gensim

from gensim.test.utils import datapath
from gensim import utils
import gensim.models

# NLTK

from nltk import pos_tag

# Misc

import random
from IPython.display import clear_output

In [ ]:
class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath("lee_background.cor")
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

In [ ]:
all_vec_words = list(model.wv.vocab.keys())
num_vec_words = len(all_vec_words)

In [ ]:
# Game constants 

NUM_WORDS = 25
NUM_GREEN_WORDS = 1
NUM_RED_WORDS = 9
NUM_BLUE_WORDS = 8
NUM_COLUMNS = 5
NUM_ROWS = 5

# Color constants

GREEN = "green"
RED = "red"
BLUE = "blue"
BLACK = "black"

# Visual constants

WORD_WIDTH  = 15
ROW_PADDING = 1

In [ ]:
# String helpers

def paint(text, colorNum):
    return "\x1b[{}m{}\x1b[0m".format(colorNum, text)

red = lambda text: paint(text, 91)
blue = lambda text: paint(text, 94)
green = lambda text: paint(text, 92)
black = lambda text: text
gray = lambda text: paint(text, 37)

colorizer_map = {
    RED: red,
    BLUE: blue,
    GREEN: green,
    BLACK: black
}

pad = lambda word: word + (" " * (WORD_WIDTH - len(word)))

In [ ]:
class Word():
    
    def __init__(self, string, color):
        self.string = string
        self.color = color
        self.has_been_guessed = False
        
    def __str__(self):
        padded_string = pad(self.string)
        colorizer = gray
        if self.has_been_guessed:
            colorizer = colorizer_map[self.color]
        return colorizer(padded_string)
            
    def guess(self):
        self.has_been_guessed = True

In [ ]:
# Generate words

def get_color(wordIndex):
    if wordIndex < NUM_GREEN_WORDS:
        return GREEN
    if wordIndex < NUM_GREEN_WORDS + NUM_RED_WORDS:
        return RED
    if wordIndex < NUM_GREEN_WORDS + NUM_RED_WORDS + NUM_BLUE_WORDS:
        return BLUE
    return BLACK

def generate_words():
    words = []
    strings = []

    while len(words) < NUM_WORDS:
        index = random.randint(0, num_vec_words)
        string = all_vec_words[index]
        tags = pos_tag([string])
        
        is_good_length = len(string) > 3 and len(string) < 10
        is_noun = tags[0][1] == "NN"
        is_dup = string in strings

        if is_good_length and is_noun and not is_dup:
            color = get_color(len(words))
            words.append(Word(string, color))

    random.shuffle(words)
    return words

In [ ]:
class Game():
    
    def __init__(self):
        self.words = generate_words()
        self.brain = self.init_brain()
        
    def init_brain(self):
        red_words = []
        blue_words = []
        bad_words = []
        for word in self.words:
            if word.color == RED:
                red_words.append(word.string)
            elif word.color == BLUE:
                blue_words.append(word.string)
            else:
                bad_words.append(word.string)
         # TODO: self.brain = Brain(red_words, blue_words, bad_words)
        
    def get_word_strings(self):
        return list(map(lambda word: word.string, self.words))
                    
    def get_user_guess(self):
        guess = input()
        if guess not in self.get_word_strings():
            print("Invalid guess. Try again.")
            return self.get_user_guess()
        return guess
        
    def print_board_state(self):
        for i in range(NUM_COLUMNS):
            row = ""
            row_padding = "\n" * ROW_PADDING
            for j in range (NUM_ROWS):
                word_index = i * NUM_COLUMNS + j
                word = self.words[word_index]
                row += str(word)
            print(row + row_padding)
            
    def print_player(self, player_color):
        colorizer = colorizer_map[player_color]
        player = colorizer(player_color.upper())
        print("Player: {}".format(player))
    
    def guess_word(self, guess, player_color):
        for word in self.words:
            if word.string == guess:
                word.guess()
                return word.color == player_color
    
    def give_hint(self, player_color):
        
        # Get hint
        # TODO: hint_word, num_hinted_words = self.brain.give_hint()
        hint_word, num_hinted_words = "yote", 2
        
        # Let user guess
        num_guesses = num_hinted_words + 1
        for i in range(num_guesses):

            # Print board state
            clear_output(wait = True)
            self.print_player(player_color)
            print("Hint: {}".format(hint_word))
            print("Remaining guess: {}\n".format(num_guesses - i))
            self.print_board_state()
            
            # Get user guess
            guess = self.get_user_guess()
            is_guess_correct = self.guess_word(guess, player_color)
            if not is_guess_correct:
                break

In [ ]:
# Play the game

game = Game()
players = [RED, BLUE]
turn_index = 0
while True:
    player = players[turn_index]
    game.give_hint(player)
    turn_index = (turn_index + 1) % 2